# Setup

In [390]:
import pandas as pd
import numpy as np
from numpy import nan as Nan
import xml.etree.ElementTree as ET # to read one dataset in XML format

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams

from IPython.display import Image, display

import requests
from string import digits
import wget 
import glob
import time

import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest
import patsy
import psutil

import plotly.graph_objects as go # to draw geospatial maps
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# Data Cleaning

#### 2.1

In [391]:
delay_2018_df = pd.read_csv("datasets/original/delay/2018.csv")
delay_2018_df = delay_2018_df[["ORIGIN","DEST","DEP_DELAY","TAXI_OUT","CANCELLED","DISTANCE","ARR_DELAY","TAXI_IN","DIVERTED"]] # columns of interest

In [392]:
delay_2018_df

,ORIGIN,DEST,DEP_DELAY,TAXI_OUT,CANCELLED,DISTANCE,ARR_DELAY,TAXI_IN,DIVERTED
0,EWR,DEN,-5.0,15.0,0.0,1605.0,-23.0,10.0,0.0
1,LAS,SFO,-8.0,11.0,0.0,414.0,-24.0,7.0,0.0
2,SNA,DEN,-5.0,15.0,0.0,846.0,-13.0,5.0,0.0
3,RSW,ORD,6.0,19.0,0.0,1120.0,-2.0,6.0,0.0
4,ORD,ALB,20.0,13.0,0.0,723.0,14.0,10.0,0.0
...,...,...,...,...,...,...,...,...,...
7213441,DCA,CLT,-4.0,20.0,0.0,331.0,-5.0,7.0,0.0
7213442,CLT,DFW,6.0,18.0,0.0,936.0,1.0,10.0,0.0
7213443,CLT,MEM,-5.0,36.0,0.0,511.0,11.0,4.0,0.0
7213444,CLT,RDU,23.0,11.0,0.0,130.0,14.0,4.0,0.0


First,we check the distribution of missing values:

In [393]:
delay_2018_df.isna().sum()

ORIGIN            0
DEST              0
DEP_DELAY    117234
TAXI_OUT     115830
CANCELLED         0
DISTANCE          0
ARR_DELAY    137040
TAXI_IN      119246
DIVERTED          0
dtype: int64

There are missing values for delay columns. However, we discovered that 95% of the missing value have "CANCALLED" == 1. It is logical to have missing delay value when the flight is never done. For these flight have already contributed to the cancel rate feature, thus should not have a effect on the delay. Since we are using the .mean() function, which will ignore all nan values, we left these rows unchanged. Same for diverted, when there shouldn't be a arrival delay because the flight is not arriving at the scheduled airport. We kept them because we can have a more accurate total departure/arrival number. In this way we have only 4000 rows that we can't explain the reason behind missing value. Since we have 7213446 rows in total, 4000 is a acceptable amount of missing. We also kept it for accurate total departure/arrival number.
We then calculated departure_delay, arrival_delay and other columns of interest. 

In [394]:
# DEPARTURE
unpleasant_2018_departure = pd.DataFrame()
unpleasant_2018_departure['total_departure'] = delay_2018_df.loc[:,["ORIGIN"]].groupby('ORIGIN').size()
unpleasant_2018_departure[["departure_delay_avg","departure_taxi_avg"]] = delay_2018_df.loc[:,["ORIGIN","DEP_DELAY","TAXI_OUT"]].groupby('ORIGIN').mean()
unpleasant_2018_departure['departure_cancelled_avg'] = delay_2018_df.loc[:,["ORIGIN","CANCELLED"]].groupby('ORIGIN').mean()
unpleasant_2018_departure['departure_distance_avg'] = delay_2018_df.loc[:,["ORIGIN","DISTANCE"]].groupby('ORIGIN').mean()
    
#ARRIVAL
unpleasant_2018_arrival = pd.DataFrame()
unpleasant_2018_arrival['total_arrival'] = delay_2018_df.loc[:,["DEST"]].groupby('DEST').size()
unpleasant_2018_arrival[["arrival_delay_avg","arrival_taxi_avg"]] = delay_2018_df.loc[:,["DEST","ARR_DELAY","TAXI_IN"]].groupby('DEST').mean()
unpleasant_2018_arrival['arrival_diverted_avg'] = delay_2018_df.loc[:,["DEST","DIVERTED"]].groupby('DEST').mean()
unpleasant_2018_arrival['arrival_distance_avg'] = delay_2018_df.loc[:,["DEST","DISTANCE"]].groupby('DEST').mean()

#Merge departure and arrival
delay_2018_df = unpleasant_2018_departure.merge(unpleasant_2018_arrival,left_index=True,right_index=True)
delay_2018_df.index.names = ["airport_code"]

In [395]:
delay_2018_df

,total_departure,departure_delay_avg,departure_taxi_avg,departure_cancelled_avg,departure_distance_avg,total_arrival,arrival_delay_avg,arrival_taxi_avg,arrival_diverted_avg,arrival_distance_avg
airport_code,,,,,,,,,,
ABE,4168,11.945071,15.095051,0.020873,619.573417,4165,5.558260,5.037680,0.004562,619.518367
ABI,2022,8.027259,13.506310,0.020277,158.000000,2022,5.784016,3.747347,0.000989,158.000000
ABQ,24047,8.635997,12.688534,0.009897,678.490456,24048,5.599697,5.385894,0.001747,678.586452
ABR,745,7.742198,19.377205,0.010738,257.000000,745,3.716621,4.771739,0.002685,257.000000
ABY,1018,15.052261,15.779543,0.010806,145.000000,1018,10.642137,3.673287,0.006876,145.000000
...,...,...,...,...,...,...,...,...,...,...
WYS,239,-2.485356,14.945607,0.000000,273.000000,239,3.063291,4.037657,0.008368,273.000000
XNA,12571,10.985233,15.636135,0.019728,603.077003,12574,5.628207,6.605621,0.001511,604.126929
YAK,722,-8.720506,9.373596,0.013850,205.500000,722,-3.577247,4.095506,0.009695,205.500000


In [396]:
len(unpleasant_airport_code_df)

357

#### 2.2

In [397]:
# extract X features
X = pd.DataFrame()
X[["total_departure","total_arrival", "departure_distance_avg","arrival_distance_avg","departure_taxi_avg","arrival_taxi_avg"]] \
= delay_2018_df.loc[:,["total_departure","total_arrival","departure_distance_avg","arrival_distance_avg","departure_taxi_avg","arrival_taxi_avg"]] 

In [398]:
X

,total_departure,total_arrival,departure_distance_avg,arrival_distance_avg,departure_taxi_avg,arrival_taxi_avg
airport_code,,,,,,
ABE,4168,4165,619.573417,619.518367,15.095051,5.037680
ABI,2022,2022,158.000000,158.000000,13.506310,3.747347
ABQ,24047,24048,678.490456,678.586452,12.688534,5.385894
ABR,745,745,257.000000,257.000000,19.377205,4.771739
ABY,1018,1018,145.000000,145.000000,15.779543,3.673287
...,...,...,...,...,...,...
WYS,239,239,273.000000,273.000000,14.945607,4.037657
XNA,12571,12574,603.077003,604.126929,15.636135,6.605621
YAK,722,722,205.500000,205.500000,9.373596,4.095506


#### 2.3 

In [399]:
airport_loc_df = pd.read_csv("datasets/original/airport/airports-extended.csv", names=["ID","name","city_name","country","airport_code","code4","latitude","longitude","altitude_ft","UTC_offset","DST","timezone","type","information_source"])

In [400]:
airport_loc_df.head()

,ID,name,city_name,country,airport_code,code4,latitude,longitude,altitude_ft,UTC_offset,DST,timezone,type,information_source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [401]:
def clean_city_name(input_city):
    original = input_city
    input_city = str(input_city)
    input_city = input_city.strip()
    input_city = input_city.lower()
    
    input_city = input_city.replace(".","")
    input_city = input_city.replace("\\\\","")
    input_city = input_city.replace("-"," ")
    input_city = input_city.replace(" - "," ")
    input_city = input_city.replace("saint ","st")
    input_city = input_city.replace("east ","")
    input_city = input_city.replace("west ","")
    
    input_city = input_city.translate({ord(k): None for k in digits})
    
    if ('/' in input_city):
        input_city = input_city[:input_city.find('/')] # in case city have muitiple names like "cityname1/cityname2"
    if ('(' in input_city):
        input_city = input_city[:input_city.find('(')] # Same as above
    if (',' in input_city):
        input_city = input_city[:input_city.find(',')] 
    input_city = input_city.strip()   
    if (' ' in input_city):
        temp=input_city.find(' ')
        if (temp > 2):
            input_city = input_city[:input_city.find(' ')]
        else:
            if (input_city.find(' ',temp+1) != -1):
                input_city = input_city[temp+1:input_city.find(' ',temp+1)]
            else:
                input_city = input_city[temp+1:]
    input_city = input_city.strip()
    try:
        assert len(input_city) > 2
        assert input_city.replace(" ","").replace("'","").isalpha()
    except:
        #print("This city name is prehaps incorrect: ",original,input_city,len(original))
        1+1
    return input_city

In [402]:
airport_loc_df = airport_loc_df.loc[:,["city_name","country","airport_code","code4", "latitude","longitude","altitude_ft"]] # add code4 since we need to use it to get runways data --YD 03/05
airport_loc_df = airport_loc_df[airport_loc_df["country"]=="United States"] # reduce dataframe size
airport_loc_df = airport_loc_df.loc[:,["city_name","airport_code","code4", "latitude","longitude","altitude_ft"]]# We don't need the country column again
airport_loc_df = airport_loc_df[airport_loc_df["airport_code"]!="\\N"] # remove NAN in index
airport_loc_df = airport_loc_df.set_index("airport_code")
airport_loc_df["city_name"] = airport_loc_df["city_name"].apply(clean_city_name)# clean city name

# store cleaned data
airport_loc_df.to_csv("datasets/cleaned/airport_loc.csv")

In [403]:
airport_loc_df

,city_name,code4,latitude,longitude,altitude_ft
airport_code,,,,,
BTI,barter,PABA,70.134003,-143.582001,2
K03,fort,PAWT,70.613403,-159.860001,35
LUR,cape,PALU,68.875099,-166.110001,16
PIZ,point,PPIZ,69.732903,-163.005005,22
ITO,hilo,PHTO,19.721399,-155.048004,38
...,...,...,...,...,...
XMR,cocoa,KXMR,28.467600,-80.566597,10
ZZV,zanesville,KZZV,39.944401,-81.892097,900
ENN,nenana,PANN,64.547302,-149.074005,362


#### 2.4 

In [404]:
us_cities_df = pd.read_csv("datasets/original/city/uscities.csv")

In [405]:
us_cities_df.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,South Creek,South Creek,WA,Washington,53053,Pierce,53053,Pierce,46.9994,-122.3921,2500,125.0,polygon,False,True,America/Los_Angeles,3,98580 98387 98338,1840042075
1,Roslyn,Roslyn,WA,Washington,53037,Kittitas,53037,Kittitas,47.2507,-121.0989,947,84.0,polygon,False,True,America/Los_Angeles,3,98941 98068 98925,1840019842
2,Sprague,Sprague,WA,Washington,53043,Lincoln,53043,Lincoln,47.3048,-117.9713,441,163.0,polygon,False,True,America/Los_Angeles,3,99032,1840021107
3,Gig Harbor,Gig Harbor,WA,Washington,53053,Pierce,53053,Pierce,47.3352,-122.5968,9507,622.0,polygon,False,True,America/Los_Angeles,3,98332 98335,1840019855
4,Lake Cassidy,Lake Cassidy,WA,Washington,53061,Snohomish,53061,Snohomish,48.0639,-122.0920,3591,131.0,polygon,False,True,America/Los_Angeles,3,98223 98258 98270,1840041959


In [406]:
# Kept latitude and longitude to differentiate cities with same name
us_cities_df = us_cities_df[["city","state_id","county_fips","county_name","population","lat","lng"]] 
us_cities_df = us_cities_df.rename(columns = {"city":"city_name"})                                              
us_cities_df["fips"] = us_cities_df["county_fips"]

def get_county_code(input_county):
    return int(input_county) % 1000

us_cities_df["county_fips"] = us_cities_df["county_fips"].apply(get_county_code)
us_cities_df["city_name"] = us_cities_df["city_name"].apply(clean_city_name)

In [407]:
us_cities_df

,city_name,state_id,county_fips,county_name,population,lat,lng,fips
0,south,WA,53,Pierce,2500,46.9994,-122.3921,53053
1,roslyn,WA,37,Kittitas,947,47.2507,-121.0989,53037
2,sprague,WA,43,Lincoln,441,47.3048,-117.9713,53043
3,gig,WA,53,Pierce,9507,47.3352,-122.5968,53053
4,lake,WA,61,Snohomish,3591,48.0639,-122.0920,53061
...,...,...,...,...,...,...,...,...
28884,montrose,SD,87,McCook,442,43.7002,-97.1843,46087
28885,spearfish,SD,81,Lawrence,12675,44.4912,-103.8167,46081
28886,emery,SD,61,Hanson,455,43.6020,-97.6195,46061
28887,watertown,SD,29,Codington,21837,44.9094,-97.1532,46029


#### 2.5 

In [408]:
airport_prop_df = unpleasant_airport_code_df.merge(airport_loc_df,how='inner',left_index=True,right_index=True)

city_search_df = pd.DataFrame(columns=["airport_code","state_id","county_id","city_id","fips","population"])
for ind,row in airport_prop_df.iterrows():
    city = row["city_name"]
    target_lat = row["latitude"]
    target_lng = row["longitude"]
    try:      
        target_cities = us_cities_df[us_cities_df["city_name"]==city]
        
        if not (target_cities.shape[0] == 1): # If there are multiple city with same name
            def calc_dis(input_):
                err = abs(target_lat - input_["lat"]) + abs(target_lng - input_["lng"])
                return err
            target_cities.loc[:,"error"] = (target_cities.apply(calc_dis,axis=1))
            target_city = target_cities.sort_values(by="error").iloc[0]
            
            assert target_city["error"] < 1.5 # assert the error should be <1.5 degs ~ 40 miles.
            
            target_city = target_city.drop(["error"])
        elif (target_cities.shape[0] >= 1):
            target_city = target_cities.iloc[0]
        
        county = str(target_city["county_fips"])
        if (len(county)==1):
            county = "00" + county
        elif (len(county)==2):
            county = "0" + county

        city_search_df = city_search_df.append({"airport_code":ind,"state_id":target_city["state_id"],"county_id":county,"city_id":target_city.name,"fips":target_city["fips"],"population":target_city["population"]},ignore_index=True)        
    except:
        try:
            def calc_dis(input_):
                err = abs(target_lat - input_["lat"]) + abs(target_lng - input_["lng"])
                return err
            us_cities_df_copy = us_cities_df
            us_cities_df_copy.loc[:,"error"] = (us_cities_df.apply(calc_dis,axis=1))
            target_city = us_cities_df_copy.sort_values(by="error").iloc[0]
            assert target_city["error"] < 1.5
            county = str(target_city["county_fips"])
            if (len(county)==1):
                county = "00" + county
            elif (len(county)==2):
                county = "0" + county
            city_search_df = city_search_df.append({"airport_code":ind,"state_id":target_city["state_id"],"county_id":county,"city_id":target_city.name,"fips":target_city["fips"],"population":target_city["population"]},ignore_index=True)
        except:
            print("No data for ",city)
            city_search_df = city_search_df.append({"airport_code":ind,"state_id":np.nan,"county_id":np.nan,"city_id":np.nan,"fips":np.nan,"population":np.nan},ignore_index=True)

#special case for DC
for ind,row in city_search_df.iterrows():
    if (row["airport_code"]=="DCA"):
        city_search_df.iloc[ind]["state_id"]="MD"
        city_search_df.iloc[ind]["county_id"]="511"

# Remove Hawaii and Alaska
city_search_df = city_search_df[(city_search_df["state_id"]!="HI") & (city_search_df["state_id"]!="AK")]
airport_prop_df = airport_prop_df.merge(city_search_df.set_index("airport_code").loc[:,["city_id","fips","population"]],left_index=True,right_index=True)

In [409]:
city_search_df

,airport_code,state_id,county_id,city_id,fips,population
0,ABE,PA,077,10988,42077,682899
1,ABI,TX,441,5333,48441,114964
2,ABQ,NM,001,3742,35001,758523
3,ABR,SD,013,28674,46013,28264
4,ABY,GA,095,17309,13095,90515
...,...,...,...,...,...,...
340,VLD,GA,185,17227,13185,79294
341,VPS,FL,091,2510,12091,4946
343,WYS,MT,031,26356,30031,1365
344,XNA,AR,007,15451,5007,49298


In [410]:
airport_prop_df # prop: property

,city_name,code4,latitude,longitude,altitude_ft,city_id,fips,population
airport_code,,,,,,,,
ABE,allentown,KABE,40.652100,-75.440804,393,10988,42077,682899
ABI,abilene,KABI,32.411301,-99.681900,1791,5333,48441,114964
ABQ,albuquerque,KABQ,35.040199,-106.609001,5355,3742,35001,758523
ABR,aberdeen,KABR,45.449100,-98.421799,1302,28674,46013,28264
ABY,albany,KABY,31.535500,-84.194504,197,17309,13095,90515
...,...,...,...,...,...,...,...,...
VLD,valdosta,KVLD,30.782499,-83.276703,203,17227,13185,79294
VPS,valparaiso,KVPS,30.483200,-86.525398,87,2510,12091,4946
WYS,yellowstone,KWYS,44.688400,-111.117996,6649,26356,30031,1365


#### 2.6 

In [411]:
def download_climate_data(state,county,year):
    save_path = "datasets/original/weather/"
    fname = state + county + "_" + str(year) + ".csv"
    if (len(glob.glob(save_path + fname))==0):
        URL = "https://www.ncdc.noaa.gov/cag/county/time-series/{}-{}-{}-all-1-2000-2020.csv?base_prd=true&begbaseyear=1901&endbaseyear=2000".format(state,county,"tavg")
        r = requests.get(URL)
        file = wget.download(URL,out=save_path + "tavg/tavg_" + fname)
        URL = "https://www.ncdc.noaa.gov/cag/county/time-series/{}-{}-{}-all-1-2000-2020.csv?base_prd=true&begbaseyear=1901&endbaseyear=2000".format(state,county,"pcp")
        r = requests.get(URL)
        file = wget.download(URL,out=save_path + "pcp/pcp_" + fname)

        tavg_df = pd.read_csv(save_path + "tavg/tavg_" + fname).iloc[4:]
        tavg_df.columns=["date","tavg","comp"]
        tavg = tavg_df.set_index("date")["tavg"]

        pcp_df = pd.read_csv(save_path + "pcp/pcp_" + fname).iloc[4:]
        pcp_df.columns=["date","pcp","comp"]
        pcp = pcp_df.set_index("date")["pcp"]

        pd.concat([tavg, pcp], axis=1).to_csv(save_path + fname)
        time.sleep(1) # not requesting too frequently

In [412]:
counter = 0
# Download data according to city_search_df
for ind,row in city_search_df.iterrows():
    try:
        download_climate_data(row["state_id"],row["county_id"],2018)
#        download_climate_data(row["state_id"],row["county_id"],2019)
    except:
        print(row)
    counter+=1
    print("progress: {:.2f}%   Just done: {}".format(100 * counter / city_search_df.shape[0],row["airport_code"]),end="\r")

# Process downloaded data
temp_pcp_df = pd.DataFrame(columns=["airport_code","temp_avg","pcp_avg"])
for ind,row in city_search_df.iterrows():
    state = row["state_id"]
    county = row["county_id"]
    save_path = "datasets/original/weather/"
    
    try:
        fname = state + county + "_2018.csv" # change it to 2018 only --YD
        temp_pcp = pd.read_csv(save_path + fname)
        tavg = temp_pcp.mean()["tavg"]
        pcp = temp_pcp.mean()["pcp"]
        
        temp_pcp_df = temp_pcp_df.append({"airport_code":row["airport_code"],"temp_avg":tavg,"pcp_avg":pcp},ignore_index=True)
        
    except:
        temp_pcp_df = temp_pcp_df.append({"airport_code":row["airport_code"],"temp_avg":np.nan,"pcp_avg":np.nan},ignore_index=True)

In [413]:
airport_prop_df = airport_prop_df.merge(temp_pcp_df.set_index("airport_code"),left_index=True,right_index=True)
airport_prop_df

,city_name,code4,latitude,longitude,altitude_ft,city_id,fips,population,temp_avg,pcp_avg
airport_code,,,,,,,,,,
ABE,allentown,KABE,40.652100,-75.440804,393,10988,42077,682899,51.902490,4.161037
ABI,abilene,KABI,32.411301,-99.681900,1791,5333,48441,114964,64.488797,2.131411
ABQ,albuquerque,KABQ,35.040199,-106.609001,5355,3742,35001,758523,54.316598,0.928465
ABR,aberdeen,KABR,45.449100,-98.421799,1302,28674,46013,28264,43.304149,1.796141
ABY,albany,KABY,31.535500,-84.194504,197,17309,13095,90515,66.578008,4.069336
...,...,...,...,...,...,...,...,...,...,...
VLD,valdosta,KVLD,30.782499,-83.276703,203,17227,13185,79294,66.983817,4.138382
VPS,valparaiso,KVPS,30.483200,-86.525398,87,2510,12091,4946,67.168050,5.020456
WYS,yellowstone,KWYS,44.688400,-111.117996,6649,26356,30031,1365,40.186307,2.004440


#### 2.7 

In [414]:
bird_strike_df = pd.read_excel("datasets/original/airport/Bird Strikes.xlsx") #data6
airport_name_df = pd.read_excel("datasets/original/airport/airportcode.xlsx") #data7 

In [415]:
bird_strike_df.head()

,Record ID,Aircraft: Type,Airport: Name,Altitude bin,Aircraft: Make/Model,Wildlife: Number struck,Wildlife: Number Struck Actual,Effect: Impact to flight,FlightDate,Effect: Indicated Damage,...,Remains of wildlife sent to Smithsonian,Remarks,Wildlife: Size,Conditions: Sky,Wildlife: Species,Pilot warned of birds or wildlife?,Cost: Total $,Feet above ground,Number of people injured,Is Aircraft Large?
0,202152,Airplane,LAGUARDIA NY,> 1000 ft,B-737-400,Over 100,859,Engine Shut Down,2000-11-23,Caused damage,...,False,FLT 753. PILOT REPTD A HUNDRED BIRDS ON UNKN T...,Medium,No Cloud,Unknown bird - medium,N,30736,1500.0,0,Yes
1,208159,Airplane,DALLAS/FORT WORTH INTL ARPT,< 1000 ft,MD-80,Over 100,424,None,2001-07-25,Caused damage,...,False,102 CARCASSES FOUND. 1 LDG LIGHT ON NOSE GEAR ...,Small,Some Cloud,Rock pigeon,Y,0,0.0,0,No
2,207601,Airplane,LAKEFRONT AIRPORT,< 1000 ft,C-500,Over 100,261,None,2001-09-14,No damage,...,False,FLEW UNDER A VERY LARGE FLOCK OF BIRDS OVER AP...,Small,No Cloud,European starling,N,0,50.0,0,No
3,215953,Airplane,SEATTLE-TACOMA INTL,< 1000 ft,B-737-400,Over 100,806,Precautionary Landing,2002-09-05,No damage,...,False,"NOTAM WARNING. 26 BIRDS HIT THE A/C, FORCING A...",Small,Some Cloud,European starling,Y,0,50.0,0,Yes
4,219878,Airplane,NORFOLK INTL,< 1000 ft,CL-RJ100/200,Over 100,942,None,2003-06-23,No damage,...,False,NO DMG REPTD.,Small,No Cloud,European starling,N,0,50.0,0,No


In [416]:
bird_strike_df = bird_strike_df[["Airport: Name", "Effect: Indicated Damage"]]
bird_strike_df = bird_strike_df.rename(columns = {"Airport: Name": "airport_name", "Effect: Indicated Damage":"bird_strike_effect"})
bird_strike_df = bird_strike_df.dropna()
bird_strike_df = bird_strike_df.reset_index(drop = True)
bird_strike_df.head()

,airport_name,bird_strike_effect
0,LAGUARDIA NY,Caused damage
1,DALLAS/FORT WORTH INTL ARPT,Caused damage
2,LAKEFRONT AIRPORT,No damage
3,SEATTLE-TACOMA INTL,No damage
4,NORFOLK INTL,No damage


In [417]:
airport_name_df.head()

,airport_code,airport_name
0,ABE,Lehigh Valley International
1,ABI,Abilene Regional Airport
2,ABQ,Albuquerque International Sunport
3,ABY,Southwest Georgia Regional
4,ACK,Nantucket Memorial


In [418]:
airport_name_df = airport_name_df.dropna()
airport_name_df = airport_name_df.reset_index(drop = True)
airport_name_df.head()

,airport_code,airport_name
0,ABE,Lehigh Valley International
1,ABI,Abilene Regional Airport
2,ABQ,Albuquerque International Sunport
3,ABY,Southwest Georgia Regional
4,ACK,Nantucket Memorial


In [419]:
def standardize_airport_name(string):
    
    string = string.lower()
    string = string.strip()
    if 'intl' in string:
        string = string.replace('intl', '')
    if 'arpt' in string:
        string = string.replace('arpt', '')
    if 'regional' in string:
        string = string.replace('regional', '')
    if 'airport' in string:
        string = string.replace('airport', '')
    if 'sunport' in string:
        string = string.replace('sunport', '')
    if 'international' in string:
        string = string.replace('international', '')
    if 'intercontinental' in string:
        string = string.replace('intercontinental', '')
    else:
        output = string
        
    string = string = string.strip()
    
    return string

In [420]:
bird_strike_df['airport_name'] = bird_strike_df['airport_name'].apply(standardize_airport_name)
airport_name_df['airport_name'] = airport_name_df['airport_name'].apply(standardize_airport_name)

def check_strike (string):
    return 1

def check_damage (string):
    if 'Caused' in string:
        output = 1
    else:
        output = 0
    return output

bird_strike_df['strike'] = bird_strike_df['bird_strike_effect'].apply(check_strike)
bird_strike_df['damage'] = bird_strike_df['bird_strike_effect'].apply(check_damage)
bird_strike_df.head()

,airport_name,bird_strike_effect,strike,damage
0,laguardia ny,Caused damage,1,1
1,dallas/fort worth,Caused damage,1,1
2,lakefront,No damage,1,0
3,seattle-tacoma,No damage,1,0
4,norfolk,No damage,1,0


In [421]:
grouped_strike = bird_strike_df.groupby('airport_name').agg({'strike':['sum']})
grouped_strike = grouped_strike.reset_index()
grouped_damage = bird_strike_df.groupby('airport_name').agg({'damage':['sum']})

bird_strike_sum_df = pd.merge(grouped_strike, grouped_damage, on='airport_name')
bird_strike_sum_df.columns = ['airport_name', 'strike_sum','damage_sum']

bird_strike_avg_df = pd.merge(airport_name_df, bird_strike_sum_df, on='airport_name')

def average_sum(input):
    output = input/(2011 - 2000 + 1)
    return output

bird_strike_avg_df['strike_avg'] = bird_strike_avg_df['strike_sum'].apply(average_sum)
bird_strike_avg_df['damage_avg'] = bird_strike_avg_df['damage_sum'].apply(average_sum)
bird_strike_avg_df = bird_strike_avg_df.drop(columns = ['strike_sum', 'damage_sum'])

bird_strike_avg_df= bird_strike_avg_df.drop(columns = ["airport_name"])
bird_strike_avg_df.head()

,airport_code,strike_avg,damage_avg
0,ABE,5.250000,0.166667
1,ABI,0.750000,0.083333
2,ABQ,6.833333,0.250000
3,ABY,0.416667,0.083333
4,ACK,0.583333,0.083333


We later need to merge this dataframe to `airport_prop_df`. However, not all airports have bird strike. Therefore, we used a right merge(that kept all rows from `airport_prop_df`) and filled nan with 0. 

In [422]:
bird_strike_final_df = pd.merge(bird_strike_avg_df, unpleasant_airport_code_df.reset_index(), how='right')
# unpleasant_airport_code_df have the same index as airport_prop_df
bird_strike_final_df = bird_strike_final_df.fillna(0)
bird_strike_final_df = bird_strike_final_df.set_index("airport_code")
bird_strike_final_df.head()

,strike_avg,damage_avg
airport_code,,
ABE,5.250000,0.166667
ABI,0.750000,0.083333
ABQ,6.833333,0.250000
ABY,0.416667,0.083333
ACK,0.583333,0.083333


#### 2.8 

In [423]:
airport_runways_df=pd.read_csv("datasets/original/airport/runways.csv")

airport_runways_df.head()

,id,airport_ref,airport_ident,length_ft,width_ft,surface,lighted,closed,le_ident,le_latitude_deg,le_longitude_deg,le_elevation_ft,le_heading_degT,le_displaced_threshold_ft,he_ident,he_latitude_deg,he_longitude_deg,he_elevation_ft,he_heading_degT,he_displaced_threshold_ft
0,269408,6523,00A,80.0,80.0,ASPH-G,1,0,H1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,255155,6524,00AK,2500.0,70.0,GRVL,0,0,N,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN,NaN
2,254165,6525,00AL,2300.0,200.0,TURF,0,0,01,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN
3,270932,6526,00AR,40.0,40.0,GRASS,0,0,H1,NaN,NaN,NaN,NaN,NaN,H1,NaN,NaN,NaN,NaN,NaN
4,322128,322127,00AS,1450.0,60.0,Turf,0,0,1,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN


In [424]:
airport_runways_df = airport_runways_df.rename(columns = {"airport_ident":"code4"})
airport_runways_df = pd.merge(airport_runways_df, airport_loc_df.reset_index(), how = 'inner', on = 'code4')
airport_runways_df = airport_runways_df[["code4", "airport_code", "length_ft", "width_ft"]]
airport_runways_df.head()

,code4,airport_code,length_ft,width_ft
0,07FA,OCA,4500.0,70.0
1,5A8,WKK,2040.0,80.0
2,HI07,WKL,26.0,26.0
3,K0V4,0V4,3798.0,60.0
4,K19S,19S,2300.0,100.0


In [425]:
runways_sum_df = airport_runways_df.groupby(['airport_code']).sum()
runways_mean_df = airport_runways_df.groupby(['airport_code']).mean()
runways_count_df = airport_runways_df.groupby(['airport_code']).count()
runway_final_df = runways_sum_df["length_ft"].to_frame().join(runways_mean_df["width_ft"].to_frame())
runway_final_df["count"] = runways_count_df["length_ft"]
runway_final_df.columns = ["length_ft_sum", "width_ft_avg","runway_count"]

runway_final_df.head()

,length_ft_sum,width_ft_avg,runway_count
airport_code,,,
0V4,3798.0,60.000000,1
19S,6800.0,80.000000,2
23M,3200.0,60.000000,1
2A5,3800.0,40.000000,1
2H0,9948.0,158.333333,3


#### 2.9 

In [426]:
enplanements_df = pd.read_excel('datasets/original/city/commercial_service_enplanements.xlsx')

In [427]:
enplanements_df.head()

,Rank,RO,ST,Locid,City,Airport Name,S/L,Hub,CY 18 Enplanements,CY 17 Enplanements,% Change
0,1.0,SO,GA,ATL,Atlanta,Hartsfield - Jackson Atlanta International,P,L,51865797.0,50251964.0,0.0321
1,2.0,WP,CA,LAX,Los Angeles,Los Angeles International,P,L,42624050.0,41232432.0,0.0338
2,3.0,GL,IL,ORD,Chicago,Chicago O'Hare International,P,L,39873927.0,38593028.0,0.0332
3,4.0,SW,TX,DFW,Fort Worth,Dallas-Fort Worth International,P,L,32821799.0,31816933.0,0.0316
4,5.0,NM,CO,DEN,Denver,Denver International,P,L,31362941.0,29809097.0,0.0521


In [428]:
enplanements_df = enplanements_df.rename(columns = {"Locid":"airport_code","CY 18 Enplanements":"enplanements"})
enplanements_df = enplanements_df.set_index("airport_code")
enplanements_df = enplanements_df[["enplanements"]] # change it to only 2018 --YD

enplanements_df.head()

,enplanements
airport_code,
ATL,51865797.0
LAX,42624050.0
ORD,39873927.0
DFW,32821799.0
DEN,31362941.0


#### 2.10 

In [429]:
security_df = pd.read_excel("datasets/original/airport/security_wait_times.xls")

In [430]:
security_df = security_df.fillna(0)
security_df = security_df.rename(columns={"Code":"airport_code"})
security_df

,airport_code,Airport,Checkpoint,2015 December,2014 December,2013 December,2012 December,2011 December,2010 December,2009 December,...,2015 January,2014 January,2013 January,2012 January,2011 January,2010 January,2009 January,2008 January,2007 January,2006 January
0,ABE,Lehigh Valley International,ABE Main Terminal,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
1,ABI,Abilene Regional Airport,ABIA,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,ABQ,Albuquerque International Sunport,Checkpoint for A/B Gates,10.0,0.0,0.0,0.0,0.0,1.0,2.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
3,ABY,Southwest Georgia Regional,ABY-A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,ACK,Nantucket Memorial,Main Terminal - 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,WYS,West Yellowstone,WYSZ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
755,XNA,Northwest Arkansas Regional,Checkpoint 2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
756,YAK,Yakutat,YAK-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
757,YKM,Yakima Air Terminal,YKM Main,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [431]:
security_df["security_avg"] = 0.0

In [432]:
def calculate_security_avg(df):
    for index,row in df.iterrows():
        row_sub = row[3:128]
        time = 0
        sums = 0
        for i in row_sub:
            if i>0:
                time = time + 1
                sums = sums + i
        df.at[index,"security_avg"] = sums/time
    return df

In [433]:
security_df = calculate_security_avg(security_df)
security_df = security_df[["airport_code", "security_avg"]].groupby("airport_code").sum()
security_df = security_df.drop(0)
security_df

,security_avg
airport_code,
ABE,2.450000
ABI,1.416667
ABQ,2.974359
ABY,1.583333
ACK,1.500000
...,...
WYS,3.428571
XNA,1.555556
YAK,3.444444


In [434]:
security_df = pd.concat([security_df, unpleasant_airport_code_df], axis=1, sort=False)
security_df = security_df.dropna()
security_df

,security_avg
ABE,2.450000
ABI,1.416667
ABQ,2.974359
ABY,1.583333
ACK,1.500000
...,...
WYS,3.428571
XNA,1.555556
YAK,3.444444
YKM,1.666667


### Final Cleaning step: merging all dataframes together to get X and Y.

In [435]:
merged_X_df = pd.concat([X,airport_prop_df, bird_strike_final_df, enplanements_df, runway_final_df, security_df], axis=1, sort=False)
merged_X_df = merged_X_df.dropna()
merged_X_df

,total_departure,total_arrival,departure_distance_avg,arrival_distance_avg,departure_taxi_avg,arrival_taxi_avg,city_name,code4,latitude,longitude,...,population,temp_avg,pcp_avg,strike_avg,damage_avg,enplanements,length_ft_sum,width_ft_avg,runway_count,security_avg
ABE,4168.0,4165.0,619.573417,619.518367,15.095051,5.037680,allentown,KABE,40.652100,-75.440804,...,682899,51.902490,4.161037,5.250000,0.166667,376468.0,13397.0,150.000000,2.0,2.450000
ABI,2022.0,2022.0,158.000000,158.000000,13.506310,3.747347,abilene,KABI,32.411301,-99.681900,...,114964,64.488797,2.131411,0.750000,0.083333,77229.0,18078.0,133.333333,3.0,1.416667
ABQ,24047.0,24048.0,678.490456,678.586452,12.688534,5.385894,albuquerque,KABQ,35.040199,-106.609001,...,758523,54.316598,0.928465,6.833333,0.250000,2647269.0,39793.0,150.000000,4.0,2.974359
ABY,1018.0,1018.0,145.000000,145.000000,15.779543,3.673287,albany,KABY,31.535500,-84.194504,...,90515,66.578008,4.069336,0.416667,0.083333,39422.0,11801.0,150.000000,2.0,1.583333
ACK,972.0,972.0,222.881687,222.881687,15.843116,6.231915,nantucket,KACK,41.253101,-70.060204,...,201,50.793361,3.703195,0.583333,0.083333,126610.0,12999.0,100.000000,3.0,1.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UIN,937.0,937.0,217.005336,217.005336,14.801386,4.159770,quincy,KUIN,39.942699,-91.194603,...,44860,53.097095,3.277469,0.000000,0.000000,11633.0,17622.0,150.000000,3.0,1.000000
VLD,1039.0,1039.0,208.000000,208.000000,16.580583,4.134951,valdosta,KVLD,30.782499,-83.276703,...,79294,66.983817,4.138382,0.250000,0.166667,44609.0,17236.0,108.333333,3.0,1.000000
VPS,7830.0,7831.0,487.504981,487.547823,14.938339,8.221585,valparaiso,KVPS,30.483200,-86.525398,...,4946,67.168050,5.020456,0.000000,0.000000,683759.0,22017.0,300.000000,2.0,3.033333
WYS,239.0,239.0,273.000000,273.000000,14.945607,4.037657,yellowstone,KWYS,44.688400,-111.117996,...,1365,40.186307,2.004440,0.000000,0.000000,8200.0,8399.0,150.000000,1.0,3.428571


In [436]:
merged_X_df.columns

Index(['total_departure', 'total_arrival', 'departure_distance_avg',
       'arrival_distance_avg', 'departure_taxi_avg', 'arrival_taxi_avg',
       'city_name', 'code4', 'latitude', 'longitude', 'altitude_ft', 'city_id',
       'fips', 'population', 'temp_avg', 'pcp_avg', 'strike_avg', 'damage_avg',
       'enplanements', 'length_ft_sum', 'width_ft_avg', 'runway_count',
       'security_avg'],
      dtype='object')

In [439]:
Y = delay_2018_df[["departure_delay_avg", "arrival_delay_avg", "departure_cancelled_avg", "arrival_diverted_avg"]]
Y = Y.merge(merged_X_df[[]],how="right",left_index=True,right_index=True)
Y

,departure_delay_avg,arrival_delay_avg,departure_cancelled_avg,arrival_diverted_avg
ABE,11.945071,5.558260,0.020873,0.004562
ABI,8.027259,5.784016,0.020277,0.000989
ABQ,8.635997,5.599697,0.009897,0.001747
ABY,15.052261,10.642137,0.010806,0.006876
ACK,23.306624,6.503219,0.036008,0.010288
...,...,...,...,...
UIN,12.049654,13.301038,0.075774,0.017076
VLD,6.724680,2.860784,0.008662,0.001925
VPS,7.357059,5.368490,0.020434,0.001660
WYS,-2.485356,3.063291,0.000000,0.008368


In [440]:
merged_X_df.to_csv("datasets/merged/merged_X.csv")
Y.to_csv("datasets/merged/Y.csv")